In [1]:
import pandas as pd
import numpy as np
from mainsequence.tdag.assets.models import Asset, ModelList
from mainsequence.tdag.contrib.benchmark_factory import MarketCapBenchmarkBars, MarketCapBenchmarkWeights
from mainsequence.tdag import AssetORM,TimeSeriesOGM
from mainsequence.tdag.time_series.update_utils import rebuild_and_update_from_source
import matplotlib.pyplot as plt
import missingno as msno
%matplotlib inline

Benchmark Configuration

In [2]:
#Benchmark configuration

REBALANCE_CONFIG = {"last_weekday_of_month": {"target_day": 4},
                            "at_time_intervals": {"time_intervals": ["12:00", "10:00"]}}
SYMBOL_LIST=["BTC","ETH"]
CASH_ASSET_SYMBOL="USDT"
ASSETS_DB="DEEPMACHINE"
cash_balance=.02 # cash that must be maitained .
frequency_id="1min"
orm = AssetORM(assets_db="DEEPMACHINE")
ogm=TimeSeriesOGM()

In [3]:

with orm.session_scope() as s:
    all_assets = s.query(Asset).filter(Asset.symbol.in_(SYMBOL_LIST), Asset.currency == "usdt").all()
    cash_asset = s.query(Asset).filter(Asset.symbol == CASH_ASSET_SYMBOL, Asset.currency == "usd").one()
    all_assets = ModelList(all_assets)
    # ts=MarketCapBenchmarkWeights(asset_list=all_assets,cash_balance=cash_balance,
    #                               cash_asset=cash_asset,frequency_id=frequency_id,
    #                               source="binance",rebalance_rule_config=rebalance_config,
    #                                    local_data_path=BASEDIR+"/data")
    benchmark_ts = MarketCapBenchmarkBars(asset_list=all_assets, cash_balance=cash_balance,
                                cash_asset=cash_asset, frequency_id=frequency_id,limit_assets=None,
                                source="binance", rebalance_rule_config=REBALANCE_CONFIG,
                                local_data_path=ogm.temp_folder)
    rebuild_and_update_from_source(assets_db=ASSETS_DB, update_tree=False, update_tree_kwargs={"DEBUG": True},
                                       serie_data_folder=benchmark_ts.data_folder)

    #get data
    benchmark_df=benchmark_ts.pandas_df

    data_df=benchmark_ts.weights_ts.daily_ts.pandas_df_concat_on_rows_by_key
    data_df=data_df.rename(columns={"key":"asset_symbol"}).set_index(["asset_symbol"],append=True)
    

NAN Analysis


In [4]:

prices_df=benchmark_ts.weights_ts.prices_df

In [5]:
msno.bar(prices_df)

In [6]:
msno.matrix(prices_df)

In [7]:
msno.matrix(benchmark_df)

In [8]:
daily_prices=prices_df/prices_df.iloc[0]
daily_prices=daily_prices.resample("1d").last()
daily_prices["benchmark"]=benchmark_df.resample("1d").last().loc[daily_prices.index]["close"]
daily_prices=daily_prices/daily_prices.iloc[0]
daily_prices.plot(figsize=(15,10))


Weights


In [9]:
w_df=prices_df=benchmark_ts.weights_ts.weights_df
w_df.resample("1h").last().plot()